Weather data preparation 

In [0]:
import pandas as pd
import numpy as np

In [0]:
#Works on a single county basis
county_name = 'NEW YORK'

In [36]:
#Read master csv file considering first column in the dataset as the index column
df_master = pd.read_csv("master.csv", index_col=0)

#df_master = df_master.drop(['County Centroid', 'Disease_INFLUENZA_A', 'Disease_INFLUENZA_UNSPECIFIED'], axis=1)   #Serves no purpose - drop the columns just for accomodating all the required columns in the same sheet (easy for debugging)
df_master.head()

,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED,UnempRate,GDP,GDP_change,Population
0,ALBANY,36001,"(42.5882713, -73.9740136)",2009,40,10/10/2009,4,4,0,0,6.8,-,-,-
1,ALBANY,36001,"(42.5882713, -73.9740136)",2009,41,10/17/2009,11,11,0,0,6.8,-,-,-
2,ALBANY,36001,"(42.5882713, -73.9740136)",2009,42,10/24/2009,27,27,0,0,6.8,-,-,-
3,ALBANY,36001,"(42.5882713, -73.9740136)",2009,43,10/31/2009,87,86,1,0,6.8,-,-,-
4,ALBANY,36001,"(42.5882713, -73.9740136)",2009,44,11/07/2009,77,76,1,0,6.8,-,-,-


In [37]:
#Read CSV file
df_TPS = pd.read_csv("NewYork_weather.csv")
df_TPS.head()
#Fill Nan values with zeros
df_TPS = df_TPS.fillna(0)
df_TPS.head()

,STATION,NAME,DATE,PRCP,SNOW,TAVG,TMAX,TMIN
0,USW00094728,"NY CITY CENTRAL PARK, NY US",2017-01-01,0.00,0.0,0.0,48,40
1,USW00094728,"NY CITY CENTRAL PARK, NY US",2017-01-02,0.21,0.0,0.0,41,37
2,USW00094728,"NY CITY CENTRAL PARK, NY US",2017-01-03,0.58,0.0,0.0,43,39
3,USW00094728,"NY CITY CENTRAL PARK, NY US",2017-01-04,0.00,0.0,0.0,52,34
4,USW00094728,"NY CITY CENTRAL PARK, NY US",2017-01-05,0.00,0.0,0.0,34,27


In [38]:
#Convert df_TPS['DATE'] to pandas Date format
df_TPS['Date'] = pd.to_datetime(df_TPS['DATE'])
#Extract Week number from date
df_TPS['Week_Number'] = df_TPS['Date'].dt.week
#Extract Year number from date
df_TPS['Year'] = df_TPS['Date'].dt.year

#Create a dict to denote how a feature has to be grouped 
#Eg: PRCP : mean - PRCP features are averaged over complete week
agg_dict = {'PRCP'  : 'mean',
            'SNOW'  : 'mean',
            'TAVG'  : 'mean', 
            'TMAX'  : 'mean',
            'TMIN'  : 'mean'}

#Group same Year and WeekNumnber as per the values in the agg_dict
df_TPS = df_TPS.groupby(['Year','Week_Number']).agg(agg_dict)
#Year and Week_NUmber are in index format, convert them into columns
df_TPS.reset_index(inplace=True)
#Take mean of TMAX and TMIN to get TAVG (TMAX and TMIN here are already averages of the entire week)
df_TPS['TAVG'] = (df_TPS['TMAX'] + df_TPS['TMIN'])/2
df_TPS.head()

,Year,Week_Number,PRCP,SNOW,TAVG,TMAX,TMIN
0,2017,1,0.165714,0.900000,32.285714,36.285714,28.285714
1,2017,2,0.098571,0.085714,38.214286,45.857143,30.571429
2,2017,3,0.080000,0.000000,41.857143,45.285714,38.428571
3,2017,4,0.337143,0.000000,40.642857,44.857143,36.428571
4,2017,5,0.008571,0.142857,33.785714,39.000000,28.571429


In [0]:
#Creates a new dataframe by merging only the required county in master dataframe and the df_TPS dataframe for that county
#This new dataframe is only a partial dataframe that contains only the rows that have the mentioned county values
#Complete dataset has to be created by either merging or creating df_TPS for all counties and cerging all temp_df
temp_df = pd.merge(df_master.loc[df_master['County'] == county_name], df_TPS,  how='left', left_on=['year','CDC Week'], right_on = ['Year','Week_Number'])

In [40]:
#Drop the newly created Year and Week_Number columns as our master dataset already has these columns
temp_df = temp_df.drop(['Year', 'Week_Number'], axis = 1)
temp_df.sample(10)

,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED,UnempRate,GDP,GDP_change,Population,PRCP,SNOW,TAVG,TMAX,TMIN
232,NEW YORK,36061,"(40.7831, -73.9712)",2016,41,10/15/2016,1,0,1,0,4.6,-,-,1636261.0,NaN,NaN,NaN,NaN,NaN
209,NEW YORK,36061,"(40.7831, -73.9712)",2015,51,12/26/2015,1,1,0,0,4.0,629682050.0,2.4,1636713.0,NaN,NaN,NaN,NaN,NaN
85,NEW YORK,36061,"(40.7831, -73.9712)",2012,7,02/18/2012,12,11,0,1,8.0,610107941.0,-,1624039.0,NaN,NaN,NaN,NaN,NaN
226,NEW YORK,36061,"(40.7831, -73.9712)",2016,16,04/23/2016,115,37,72,6,4.6,-,-,1636261.0,NaN,NaN,NaN,NaN,NaN
57,NEW YORK,36061,"(40.7831, -73.9712)",2011,13,04/02/2011,16,5,11,0,7.8,-,-,1608215.0,NaN,NaN,NaN,NaN,NaN
92,NEW YORK,36061,"(40.7831, -73.9712)",2012,14,04/07/2012,16,15,1,0,8.0,610107941.0,-,1624039.0,NaN,NaN,NaN,NaN,NaN
323,NEW YORK,36061,"(40.7831, -73.9712)",2019,13,03/30/2019,165,152,13,0,-,-,-,-,0.008571,0.000000,48.928571,57.000000,40.857143
117,NEW YORK,36061,"(40.7831, -73.9712)",2013,6,02/09/2013,124,76,44,4,7.5,600904801.0,-1.5,1627806.0,NaN,NaN,NaN,NaN,NaN
282,NEW YORK,36061,"(40.7831, -73.9712)",2018,5,02/03/2018,900,493,384,23,3.7,-,-,1628701.0,0.185714,0.142857,32.214286,39.857143,24.571429
70,NEW YORK,36061,"(40.7831, -73.9712)",2011,45,11/12/2011,2,2,0,0,7.8,-,-,1608215.0,NaN,NaN,NaN,NaN,NaN
